<a href="https://colab.research.google.com/github/sanket1009/dsrs-kb-prototype/blob/main/DSRS_KB_Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DSRS Knowledge-Base Prototype (Screening Task – Job ID 5)
## 0. Executive summary
## 1. Data & content acquisition
## 2. Initial KB build
## 3. Exploration & visual demo
## 4. Proposed schema & features
## 5. Maintenance & growth roadmap
## 6. References

In [ ]:
!pip install --quiet pandas networkx pyvis


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.9 MB/s eta 0:00:00
